In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.model_selection import train_test_split, KFold, cross_validate, ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostRegressor
import re
import statsmodels.api as sm

In [2]:
X_train = pd.read_csv('data/X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('data/X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16570 entries, 0 to 16569
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   직종      16570 non-null  object 
 1   세부직종    16570 non-null  object 
 2   직무태그    14055 non-null  object 
 3   근무경력    16570 non-null  object 
 4   근무형태    6661 non-null   object 
 5   근무지역    16570 non-null  object 
 6   출신대학    16570 non-null  object 
 7   대학전공    16570 non-null  object 
 8   어학시험    4988 non-null   object 
 9   자격증     16570 non-null  object 
 10  대학성적    14600 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.4+ MB


In [4]:
X_train['NaN_count'] = X_train.isnull().sum(1)
X_test['NaN_count'] = X_test.isnull().sum(1)

In [5]:
X_train['NotNull_count'] = X_train.notnull().sum(1) -1
X_test['NotNull_count'] = X_test.notnull().sum(1) -1

In [6]:
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,대학성적,NaN_count,NotNull_count
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0개월,NaN,"서울,경기,서울",성균관대학교,국문,NaN,無,70.0,2,9
1,경영·기획·회계·사무,사무·총무·법무,NaN,2년 11개월,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,NaN,2,9


#### 근무경력-년 월 구분

In [7]:
X_train['숙련도'] = X_train['근무경력'].str[0].astype(float)
X_train['숙련도'] = X_train['숙련도'].apply(lambda x:'경력없음' 
                          if x==0 else '하' if x<=3 else '중' if x<=6 else '상')

In [8]:
X_test['숙련도'] = X_test['근무경력'].str[0].astype(float)
X_test['숙련도'] = X_test['숙련도'].apply(lambda x:'경력없음' 
                          if x==0 else '하' if x<=3 else '중' if x<=6 else '상')

In [9]:
X_train_a, X_test_a = X_train.copy(), X_test.copy()
X_all = pd.concat([X_train_a,X_test_a], ignore_index=True)

In [10]:
for i in range(len(X_all.근무경력)):
    if len(X_all.근무경력[i]) == 3 or len(X_all.근무경력[i]) == 4:
        X_all.근무경력[i] = '0년 ' + X_all.근무경력[i]

In [11]:
we = X_all.근무경력.str.split(' ',expand=True)
we.columns = ['년','개월']

In [12]:
# year, month로 구분하여 열 생성
we = X_all.근무경력.str.split(' ',expand=True)
we.columns = ['년','개월']

we_y = we.년.str.split('년', expand=True) 
we_y.columns = ['년','삭제']
we_y.drop('삭제',axis=1,inplace=True)
we_y = we_y.astype(int) 

we_m = we.개월.str.split('개월', expand=True)
we_m.columns = ['개월','삭제']
we_m.drop('삭제',axis=1,inplace=True)
we_m = we_m.astype(int)

X_all['근무경력_y'] = we_y['년']
X_all['근무경력_m'] = we_m['개월']

In [13]:
X_train = X_all.iloc[:X_train.shape[0],:].reset_index(drop=True)
X_test = X_all.iloc[X_train.shape[0]:,:].reset_index(drop=True)

In [14]:
### 경력을 수치로 변환

def f1(x):
    numbers = re.findall(r'\d+', x)
    return int(numbers[0])*12 if len(numbers) == 1 else int(numbers[0])*12+int(numbers[1])

X_train['근무경력'] = X_train['근무경력'].apply(f1)
X_test['근무경력'] = X_test['근무경력'].apply(f1)

In [15]:
X_train['신입경력'] = 1
X_train['신입경력'].loc[X_train['근무경력'] <= 12] = 0

X_test['신입경력'] = 1
X_test['신입경력'].loc[X_test['근무경력'] <= 12] = 0

#### 근무경력 제곱항 추가

In [16]:
X_train['근무경력^2'] = np.square(X_train['근무경력'])
X_test['근무경력^2'] = np.square(X_test['근무경력'])

#### 근무경력 세분화

In [17]:
def seg_worklength(x):
    if 0.0 <= x < 24.0:
        x = '0to2'
    elif 24 <= x < 48:
        x = '2to4'
    elif 48 <= x < 72: 
        x = '4to6'
    elif 72 <= x < 96:
        x = '6to8'
    elif 96 <= x < 120:
        x = '8to10'
    elif 120 <= x:
        x = 'over10'
    return x
    
X_train['근무경력_세분화'] = X_train['근무경력'].apply(seg_worklength)
X_test['근무경력_세분화'] =X_test['근무경력'].apply(seg_worklength)

####  마지막 근무형태

In [18]:
X_train_cp, X_test_cp = X_train.copy(), X_test.copy()

In [19]:
X_train_cp['근무형태'].fillna('N', inplace=True)
X_test_cp['근무형태'].fillna('N', inplace=True)

In [20]:
X_train['마지막근무형태'] = 0
for i,j in enumerate(X_train_cp['근무형태']) :
    X_train_cp['근무형태'][i] = j.rstrip(', ')
for i,j in enumerate(X_train_cp['근무형태'].str.split(', ')):
    X_train['마지막근무형태'][i] = j[-1]



X_test['마지막근무형태'] = 0
for i,j in enumerate(X_test_cp['근무형태']) :
    X_test_cp['근무형태'][i] = j.rstrip(', ')
for i,j in enumerate(X_test_cp['근무형태'].str.split(', ')):
    X_test['마지막근무형태'][i] = j[-1]

#### 해외근무지역 피쳐

In [21]:
korea = ['서울','부산', '경기', '전국', '기타', '전북', '충남', '광주', '인천', '대전', '경남','충북','울산', '경북', '강원','전남'
        ,'대구','제주','광주']
foreign = ['홍콩','미국','해외','중국','캐나다','인도네시아','일본','싱가포르','필리핀','대만','말레이시아','인도네시아','러시아','인도'
          ,'프랑스','필리핀','말레이시아','방글라데시']
world = korea + foreign

f_f = X_train.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_train.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_train.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

c_index = X_train.loc[c,'근무지역'].index

X_train['해외근무지역'] =  (X_train.index).isin(c_index)

f_f = X_test.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_test.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_test.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

ct_index = X_test.loc[c,'근무지역'].index

X_test['해외근무지역'] =  (X_test.index).isin(ct_index)

X_train['해외근무지역'] = X_train['해외근무지역'].apply(lambda x: '유' if x== True else '무')
X_test['해외근무지역'] = X_test['해외근무지역'].apply(lambda x: '유' if x== True else '무')

#### 근무지역 abc

In [22]:
a = X_train['근무지역'].str.split(',')
b = []
c = []
d = []
for i in a :
    b.append(i[0])
    c.append(i[1])    
    d.append(i[2])    

b = pd.Series(b)
c = pd.Series(c)
d = pd.Series(d)

a = pd.concat([b,c,d],axis=1)
a.columns = ['근무지역_a','근무지역_b','근무지역_c']

X_train.replace({'근무지역_c':{'':'N'},
                '근무지역_b':{'':'N'}},inplace=True)

X_train = pd.concat([X_train,a], axis=1)

In [23]:
a = X_test['근무지역'].str.split(',')
b = []
c = []
d = []
for i in a :
    b.append(i[0])
    c.append(i[1])    
    d.append(i[2])    

b = pd.Series(b)
c = pd.Series(c)
d = pd.Series(d)

a = pd.concat([b,c,d],axis=1)
a.columns = ['근무지역_a','근무지역_b','근무지역_c']

X_test.replace({'근무지역_c':{'':'N'},
                '근무지역_b':{'':'N'}},inplace=True)

X_test = pd.concat([X_test,a], axis=1)

#### 근무희망형태

In [24]:
X_train['근무희망형태'] = X_train['근무형태']
X_test['근무희망형태'] = X_test['근무형태']

X_train['근무희망형태'].fillna('unknown',inplace = True) 
X_test['근무희망형태'].fillna('unknown',inplace = True) 

In [25]:
X_train['근무희망형태'] = X_train['근무희망형태'].apply(lambda x: -1 if x.split(',')[0] == 'unknown' else len(x.split(',')))
X_test['근무희망형태'] = X_test['근무희망형태'].apply(lambda x: -1 if x.split(',')[0] == 'unknown' else len(x.split(',')))

#### 단과대

In [26]:
df_college = pd.read_csv('data/단과대_최종.csv', encoding='cp949')

In [27]:
인문대 = list(df_college['인문대'])
사과대 = list(df_college['사과대'])
경영경제대 = list(df_college['경영경제대'])
교육대 = list(df_college['교육대'])
예체능 = list(df_college['예체능'])
공대 = list(df_college['공대'])
보건계열 = list(df_college['보건계열'])
자연과학대 = list(df_college['자연과학'])
법대 = list(df_college['법대'])
소프트웨어대 = list(df_college['소프트웨어'])
농축산대 = list(df_college['농축산대'])
건축대 = list(df_college['건축대'])
신학대 = list(df_college['신학대'])
결측 = list(df_college['결측'])

In [28]:
X_train['단과대'] = X_train.대학전공.copy()
X_test['단과대'] = X_test.대학전공.copy()

In [29]:
X_train['단과대'] = X_train['단과대'].replace(인문대, '인문대')
X_train['단과대'] = X_train['단과대'].replace(사과대, '사과대')
X_train['단과대'] = X_train['단과대'].replace(경영경제대, '경영경제대')
X_train['단과대'] = X_train['단과대'].replace(교육대, '교육대')
X_train['단과대'] = X_train['단과대'].replace(예체능, '예체능')
X_train['단과대'] = X_train['단과대'].replace(공대, '공대')
X_train['단과대'] = X_train['단과대'].replace(보건계열, '보건계열')
X_train['단과대'] = X_train['단과대'].replace(자연과학대, '자연과학')
X_train['단과대'] = X_train['단과대'].replace(법대, '법대')
X_train['단과대'] = X_train['단과대'].replace(소프트웨어대, '소프트웨어대')
X_train['단과대'] = X_train['단과대'].replace(농축산대, '농축산대')
X_train['단과대'] = X_train['단과대'].replace(건축대, '건축대')
X_train['단과대'] = X_train['단과대'].replace(신학대, '신학대')
X_train['단과대'] = X_train['단과대'].replace(결측, '결측')

In [30]:
X_test['단과대'] = X_test['단과대'].replace(인문대, '인문대')
X_test['단과대'] = X_test['단과대'].replace(사과대, '사과대')
X_test['단과대'] = X_test['단과대'].replace(경영경제대, '경영경제대')
X_test['단과대'] = X_test['단과대'].replace(교육대, '교육대')
X_test['단과대'] = X_test['단과대'].replace(예체능, '예체능')
X_test['단과대'] = X_test['단과대'].replace(공대, '공대')
X_test['단과대'] = X_test['단과대'].replace(보건계열, '보건계열')
X_test['단과대'] = X_test['단과대'].replace(자연과학대, '자연과학')
X_test['단과대'] = X_test['단과대'].replace(법대, '법대')
X_test['단과대'] = X_test['단과대'].replace(소프트웨어대, '소프트웨어대')
X_test['단과대'] = X_test['단과대'].replace(농축산대, '농축산대')
X_test['단과대'] = X_test['단과대'].replace(건축대, '건축대')
X_test['단과대'] = X_test['단과대'].replace(신학대, '신학대')
X_test['단과대'] = X_test['단과대'].replace(결측, '결측')

In [31]:
X_train['단과대'].value_counts()

경영경제대     4032
공대        3160
인문대       2310
예체능       1879
소프트웨어대    1853
사과대       1245
자연과학      1036
법대         319
교육대        237
건축대        218
농축산대       111
보건계열        81
신학대         61
결측          28
Name: 단과대, dtype: int64

#### id 생성

In [32]:
X_train['ID'] = \
X_train['근무경력'].astype(str) + '_' + X_train['출신대학'].astype(str) + '_' +\
X_train['대학전공'].astype(str) + '_' + X_train['어학시험'].astype(str) + '_' +\
X_train['자격증'].astype(str) + '_' + X_train['대학성적'].astype(str) 

In [33]:
X_test['ID'] = \
X_test['근무경력'].astype(str) + '_' + X_test['출신대학'].astype(str) + '_' +\
X_test['대학전공'].astype(str) + '_' + X_test['어학시험'].astype(str) + '_' +\
X_test['자격증'].astype(str) + '_' + X_test['대학성적'].astype(str) 

#### 대학전공, 경력세분화

In [34]:
가정 = ['가정교육','가정복지','가정학', '가정학(인테리어)', '가정학부','가족경영 소비자 학과','소비자학',
 '가족경영.소비자','가족경영.소비자학과','가족문화.소비자','가족문화소비자', '가족문화소비자학',
 '가족문화소비자 학과', '가족문화와 소비자학과','가족환경학', '가족환경(가정관리)','가정학과','가정관리학과및 건축학과부전공',
'소비자아동주거학과','소비자인간발달','소비자아동학과','가정관리학','가정관리학과','가정관리','소비자인간발달학과',
      '아동가정환경학과','가정교육과','가정관리(현재아동가족학과로변경됨)','노인복지','노년학','소비자가족학과',
      '가족문화.소비자학과','가정복지학과','가족문화소비자학과']

건강 = ['간호학','건강관리전공', '건강관리학','건강스포츠과', '건강스포츠학','보건관리', '보건관리학','보건교육','보건학',
'수족병리학','보건관리학과','간호학과','건강관리학과','보건교육학','응급구조학과','보건체육','임상병리학','수족병리학과',
  '스포츠건강과학','운동심리,인체생리,운동생리','건강스포츠학과','보건학과'
       ]

건설 = ['건설공', '건설공학', '건설공학 토목','건설시스템공학','건설환경공학', '건설환경공',
'건축', '건축공', '건축과', '건축학','건축 공학과', '건축.도시공학부', '건축도시공학', '건설시스템공학과',
 '건축공', '건축공학', '건축공학부','건축설계','건축설비', '건축설비공학','건축공학과',
     '건축학과','건설환경공학과','산업건설환경공학부','건충공학','설계전공','건축설비공학과','건축설계전공','건축시공',
    '건축조경','건설공학부',
    ]


경영 = ['경영학', '경영', '경영학과', '경영학전공', '경영학 전공','경영정보학과' ,'경영학부',
'경역학과', '경영(2학년편입)', '경영/부전공:무역', '경영/회계', '경영/회계/세무', 
'경영학 (경제학 부전공)', '경영학(독학사)', '경영학과 (주간)','경영학사', 'BUSINESS ADMINISTRATION',
'경영,회계학', '경영,회계학과','경영.회계','경영회계학부', '경영 회계', '경영회계', '경영회계학', 
'경영.회계학과', '경영획계학과','경영학부(경영학,회계학)','경영,회계학', '경영,회계학과','경영.회계','경영회계학부',
 '경영 회계', '경영회계', '경영회계학', '경영.회계학과', '경영획계학과','경영학부(경영학,회계학)',
'경영경제', '경영경제 (주)', '경영경제 전공', '경영경제학', '경제 / 경영', '경제, 경영',
'경영정보','경영정보(MIS)', '경영정보과', '경영정보과학부','경영정보시스템', '경영정보학', 
'경영정보학과 편입','경영정보회계','MIS', '정보 경영학', '정보경영학부',
'경영대학원/마케팅', '마케팅', '마케팅학','마케팅학과','디지털경영', '디지털경영회계학','미술경영',
'경영대학 비서학과', '비서', '비서학','세무.회계', '세무회계', '세무회계  과수석졸업', '세무회계학',
'시스템경영공학', '시스템경영공학부','유통경영', '유통경영과','유통관리학', '유통관리','유통정보', '유통정보학', 
 '유통경영학과','유통관리학과','유통정보학과','재무금융', '재무금융학',
'경영학부(회계학)', '경영학부/회계학','경영학부/회계학전공', '경영학부회계학', 
'상경학부(회계전공)', '회계', '회계학', '회계학 졸업예정', '회계학전공', '회계학과(야)', 
'회계학과졸업', '회계학교''회계확과','회계세무','회계정보', '회계정보학','e-businees학과',
 'E-BUSINESS', 'E비즈니스', 'E-비즈니스 IT 학부', 'E비지니스','경영학부 물류유통 전문화과정',
 '경영학부 물류유통학과정', '경영(물류유통)','경영(물류,유통)', '경영학부(물류유통)', '물류유통','물류 유통','물류.유통'
,'경상 무역학과','경상학부 무역학과', '경상학부 무역과', '경상학부 무역학', '경영대학 무역학과', 
'경영학부 무역학', '경영학전공(무역학부전공)', '국제경영학부(무역학과)', '국제통상학부 무역학과',
'무역', '무역(국제경영)학', '무역전공', '무역학', '무역학과 졸업', '무역학과(경영)', '무역학부','회계학과',
'무역학전공','경영/무역','경영무역','상업교육', '상업교육과','생산가공','경영학 (복수전공)','경영, 원예','경영학/역사학',
'비서학과','경영,원예','이비지니스학과','Management',
'경영회계학과','수산경영','시스템경영','경영학부/회계학과','경영행정학부','회계정보학과','시스템경영공학과',
'이비즈니스학과','우통경영','전자상거래','경영학(물류유통)','세무회계학과','물류,품질,안전,인간공학',
'회계확과','e-Business','미술경영학','인터넷경영정보','마케팅전공','회계학교','상업교육학과','종합파이낸셜',
'Public Administration','유통학과','생산가공공학과'
]

경제 = ['경제', '경제학', '경제학부', '경제학사',
'경제/정보처리','경제,금융', '경제,금융학과', '경제금융', 'e-비즈니스학부 금융경제학과','경제,무역', '경제. 무역',
 '경제.무역', '경제무역', '경제무역과', '경제무역학', '경제통상', '경제통상학', '경제학통상학','농경제학', '농업경제학',
'상경학과', '경제,무역, 금융학부','경제학부 응용경제학전공','자원경제학','국제통상,무역','경제학과','무역학과',
    '소비자경제학','경제무역학과','중국통상','일본통상전공','농업경제학과','응용경제학','자원경제학과','정치경제학','응용경제학과',
    '응용경제','지역경제학',
    ]


관광 = ['경영,관광경영', '관관경영학', '관광 경영', '관광 경영학과', '관광경여', '관광경영', '관광경영학',
'관광경영학과 경영학사', '관광경영학전공', '광광경영학', '산업정보대학 관광경영학과', 
'경상복지대학 관광학과', '관광', '관광과', '관광학', '관광학부','Tourism Management',
'경상대학 관광개발', '관광개발', '관광개발과', '관광개발학',
'관광영어','관광영어과', '관광영어통역학와', '관광외국어 학부, 영어과',
'관광일본어', '관광일어', '관광일어과','관광통역', '관광통역학', '관광통역학과 중국어','관광경영학과','관광학과',
'관광개발학과','관광조경학','관광레저학부','관광정보학','관광영어학과','관광일어학과','관광통역학과','관광행정학과'
    ]



교육 = ['교육', '교육학', '교육학과(영어부전공)','교육공학','교육심리', '교육심리학',
      '교정학','평생교육학','교육/영문','사범','교정학과','미&#49715;교육과','평생교육학과',
     '교육공학과','특수교육','교육학과','교육심리학과'
    ]

국제 = ['국제', '국제학', '국제학부', '국제학부(중국학)','국제학부러시아학','국제 물류과', '국제 물류학과',
 '국제 통상', '국제무역', '국제무역학과', '국제물류학과', 
'국제통상', '국제통상 학과', '국제통상학', '국제통상학과(무역학과)','국제경영', '국제경영학','국제경제', '국제경제학',
'국제 관광과','국제관광', '국제관광학', '국제관광경영학과', '국제관광통상학부','국제경영학과',
'국제관광경영', '국제관광', '국제관광통상학부 국제관광학', '국제관광학과','국제관계', '국제관계학','국제유엔학과',
'국제통상학과','국제관계학과','국제학과','국제경제ㆍ경영','국제경제학과','국제정경학과','국제산업정보','국제특송',
]


기계 = ['기게공학과', '기계', '기계 공학', '기계 공학과',  '기계고학부','기계/항공우주공학과',
'기계공', '기계공악', '기계공학', '기계공학 전공', '기계공학고', '기계공학과', '기계과',
 '기계공학부(기계설계)', '기계설게공학과', '기계설계', '기계설계 공 학과', '기계설계(생산시스템)', 
'기계설계공학', '기계설계과', '기계설계디자인', '기계설계학', '기계설계학과''기계시스템', '기계시스템공', 
 '기계시스템공학', '기계시스템전공','기관공학','기계및자동제어공학부', '기계및제어', 
 '기계및제어공학부','기계*자동제어학부','기계 및 제어공학부','기계공학부 자동차공학 전공','생산기계공학',
'자동차기계공학','자동차/기계','기계공학부 자동차공학 전공','자동차공학', '기계공학부','기계설계학과',
     '생산기계공학과','자동차', '기계설계공학과','기계시스템공학과','기계에너지생산공학부','기계,산업공학부',
'기계전자공학부','기계시스템','기계공확과','자동차기계공학부','기계자동차공학과','정밀기계공학','기계항공우주공학과',
'농기계학과','기계정밀','에너지공학과','메카트로닉스', '메카트로닉스 공학과', '메카트로닉스공',
      '메카트로닉스공학', '메카트로릭스공학','메카트로닉스공학과','제어계측공학과 / 전기 / 전자','제어 계측', '제어계측', '제어계측공', '제어계측공학', 
 '제어계측공학 전공', '제어계측공학전공','제어시스템공학','자동제어',]



도시 = ['도시 지역계획학과', '도시 및 지역계획', '도시계획', '도시계획학', '도시지역계획학과', '도시지역계획',
'도시 및 지역개발학과', '도시/부동산개발학과', '도시개발', '도시계발학과', '도시부동산개발', '부동산개발학과',
'도시건축공학과', '도시공학', '도시공학전공','도시조경','도시행정학',
'도시부동산학과','도시공학과','도시조경학','도시지역과학부','도시계획학과','도시행정학과','도시개발학과',]



법 = ['법', '법학', '법학(부전공)', '법학과(심화전공)', '법학부', '법학학사', '/  법 / 경영 (부전공)',
'정외과', '정치 외교 학과', '정치 외교학과', '정치외교', '정치외교학','정치행정',
'법과대학 행정학', '법정대 행정학', '법행정', '사회과학행정전공','행정',
 '행정학', '행정과', '행정학과 졸업', '행정학 전공','경찰행정', '경찰행정학', '경철행정학', '행정,경찰행정','경호학',
'행정/사회복지','행정학,경영학','법학과','행정학과','법과대학','경찰법학부','정치외교학과','경찰행정학과',
     '경찰학','행정학전공','정치행정학'
    ]

사회 = ['사회복지', '사회복지학', '사회복지과', '사회복지학부 청소년상담심리전공','사회과학',
 '사회사업', '사회사업과', '사회사업학','사회정보','사회학','사회교육', '사회교육학', 
 '사회과 교육', '공통사회교육','사회학/방송영상학/유럽학','사회학/철학','사범대 사회생활학과',
     '사회학과','사회복지학과','사회교육학과','사회과학부','사회사업학과','사회교욱','사회복지학부','사회정보학과'
    ]

산업 = ['산업건설환경공','산업공', '산업공학', '산업시스템공학', '산업공학과','산업심리', '산업심리학', 
 '산업심리학(경영학부)','경영학부/산업심리학', '경영/산업심리','산업안전공', '산업안전공학', '산업안전공학과 졸업',
 '산업재산권학', '법학,산업재산권학', '산업재산권','산업정보(경영학)', '산업정보', '산업정보공학','산업정보시스템',
 '산업정보학','산업경영', '산업 경영학과','산업경영공학', '산업경영공학부','산업경제', '산업 경제학','산업경영공학과',
      '산업시스템','산업정보공학과','산업정보시스템공학과','산업경영학과','산업환경','산업심리학과',
      '산업시스템공학과','산업정보학과','상업공학과','산업공학과 개발공학 전공'
    ,'산업교육학']


 
생명 = ['생명공학', '생명공학과','생명과학', '생명과학전공', '생명과학학과', '생명과학부','생명화학공학','생명자원',
'생물공학','생물과학','생물교육','생물기전공','생물자원', '생물자원학(농학과)', '생물자원학전공', '생물자원학',
'생물학', '생물', '생물과','유전공','유전공학','응용생물공학','응용생물과학', '응용생물', '응용생물학', '응용생물학부',
'미생물', '미생물 전공', '미생물공학과','미생물학','분자생물학과','미생물공학, 경영학과(복수전공)','생물, 영문학과',
'생물학과','식품생물공학과','유전공학과','생물,영문학과','영양학','영양학과',
      '생명의학','생명환경디자인','생명과학과','응용분자생물학','생명화학공학과','생명공학부','생물자원공학',
    '분자생물학','생명자원과학부','미생물학과','생물기전공학과','생물공학과','분자생물','응용생물공학과','생물교육과',
    '생물과학과','응용분자생물학과']




연극 = ['연극과', '연극연기', '연극영화과연기전공','연극영화-연출', '연극영화학', 'Theatre','연극영화학부 공연영상미술',
     '영화영상학과']


    
의류 = ['의류직물', '의류직물학','의류직물학 4년수료','의류패션', '의류패션산업', '의류패션전공',
 '생활과학부 의상학과', '예술대학 의상학과', '의상', '의상전공', '의상학','생활과학대학 의류학과',
 '의류','의류학', '의류헉과','의류환경''공연예술학과 의상디자인', '의상 디자인', '의상 디자인과','의상 디자인과', 
'의상 디자인학과', '의상디자인', '의상디자인과', '의상디자인전공', '의상디자인학',
 '의상디자인학과졸', '의생활학부 의상디자인','의상,섬유디자인', '의상.섬유 디자인', '의상.섬유 디자인', '의상.섬유디자인',
'의상섬유 디자인과', '의상섬유디자인','생활과학부 패션산업전공','의생활','의류직물학/소비자인간발달학(복수전공)',
    '의류환경/경영이중전공','의류환경/생활디자인','의류직물학과','의류환경',
     '의류학과','의상학과','패션산업학과','의류패션산업학과','의생활학과','전통패션문화전공','의류 패션 학과']



전기 = ['전기 공학과', '전기공', '전기공학', '전기공학전공','전기',
'전기 전자 제어', '전기전자제어', '전기전자제어공', '전기전자제어공학', '전기전자제어공학과',
'전기전자제어전공', '전자전기제어공학부','전기 전자 컴퓨터', '전기전자 및 컴퓨터 공학부','전기전자 컴퓨터공학부/컴퓨터공학과', 
'전기전자및컴퓨터공학', '전기전자및컴퓨터공학부', '전기전자컴퓨터 공학과', 
'전기전자컴퓨터공학', '전자전기컴퓨터공학과', '전자전기컴퓨터공학부','전기전자정보공학', '전기전자정보공학과', 
 '전자정보', '전자정보공학', '전자정보공학과','전자정보공학부','전기전자정보통신공학', '전자정보통신',
 '전자정보통신공', '전자정보통신공학', '전자정보통신공학부','전기,전자', '전기(전자)', '전기전자','전기전자 공학부',
 '전기전자공', '전기전자공학', '전기전자공학과', '전자전기', '전자전기공', '전자전기공학', '전자전기공학부',
 '공대 전기전자공학과','전자', '전자전공','기계전자공학부 전자 전공','전자 계산학과', '전자계산',
 '전자계산과', '전자계산학','전자 공학', '전자공', '전자공하과', '전자공학', '전자공학전공','전자계산공학',
'전자물리', '전주물리학','전자재료', '전자재료 공학', '전자재료(전기전자공학부)', '전자재료공', '전자재료공학',
'전자통신', '전자통신공','전자통신공학', '전자통신과','전파공학', '전파공학교(통신)', '전파공학전공', '전파통신공학',
'전자공학과','전자재료공학과','전파공학과','전기공학과','전자계산학과','전기전자정보',
'전기공화과','전기공학부','전자물리학','전기전자컴퓨터', '광전자공학과','전기전자제어공학부','전자통신공학과',
'전기전자공학부','전자계산공학과','전기전자컴퓨터공학부','전자과','전기과','전자기계','전기전자정보공학부','반도체',
'전기전자정보통신공학부']



정보 = ['정보공', '정보공학', '정보전산학부 정보공학','정보네트워크','정보디자인', '정보디자인과','정보미디어','정보보호',
'정보산업','정보시스템', '정보시스템 공학', '정보시스템공학','정보전산','정보전자공학','정보처리', '정보처리학',
 '정보처리학과','정보통계','정보통계학','정보통신', '정보통신학' ,'정보통신학과','정보통신 공학', '정보통신 공학과',
'정보통신공', '정보통신공학', '정보통신공학부','정보통신공학사', '정보통신공학전공', '정보보호.시스템','정보통신/전파공학',
'정보통신과', '정보통신학', '정보통신학과','정보통신전파계열','광전자공학','광통신과', '광통신공학','정보통신학과(구 전자공학과)',
 '지식 정보 산업공학과','지식정보', '지식정보학', '지식정보학부','통신공학','통신컴퓨터 공학부','정보공학과','정보통신공학과',
'정보 통신 전파 공학','정보산업공학','정보전자공학과','정보시스템공학과','정보통신전파공학','정보관리',
'정보시스탬','정보통공학과','데이터정보전공','정보네트워크공학','통신공학과'
    
    ]

  
컴퓨터 = ['컴퓨터', '정보전산학부(컴퓨터 전공)', '컴퓨터전공', '컴퓨터학', '컴퓨터학부', 
'통신`컴퓨터공학분 컴퓨터전공', '통신컴퓨터공학부 컴퓨터전공','전기전자컴퓨터공학부 컴퓨터공학전공', '컴공', '컴퓨터공학전공', '컴퓨터 공학', 
'컴퓨터 공학과', '컴퓨터 공학과 졸업', '컴퓨터공', '컴퓨터공학', '컴퓨터공학(전자계산학)','컴퓨터공학부', '컴퓨터공학전공',
'컴정', '컴퓨터 정보공학', '컴퓨터 정보학부', '컴퓨터공학, 정보공학', '컴퓨터정보', '컴퓨터정보공학',
 '컴퓨터정보공학전공', '컴퓨터정보공학부 정보통신공학전공','컴퓨터 과학', '컴퓨터 과학과', '컴퓨터과학', '컴퓨터과학학과',
'컴퓨터교육','컴퓨터그래픽디자인', '그래픽디자인','컴퓨터수리정보','컴퓨터시스템공학, 컴퓨터시스템과', '컴퓨터시스템학과',
'컴퓨터응용물리','컴퓨터전자물리', '컴퓨터전자물리학','컴퓨터정보과학','컴퓨터정보통신','컴퓨터정보학',
'전산', '전산계산', '전산과', '전산학', '전산학부','전산공', '전산공학','전산과학', '전산과학학과',
'전산정보', '전산정보학','전산통계', '전산통계학', '전산통계학교',
'소프트웨어', '소트웨어공학', '소프트웨어공학', '소프트웨어공학과', '소프트웨어시스템', 
 '소프트웨어시스템공학', '정보전산학부 소프트웨어시스템공학과','소프트웨어학' ,'정보통신공학부 소프트웨어공학',
 '정보통신공학부 소프트웨어공학과','컴퓨터학과, 물리학과','컴퓨터학과','정보전산학부','컴퓨터공학과',
'컴퓨터 응용 물리','컴퓨터정보학부','전산학과','컴퓨터과학과','컴퓨터학부 컴퓨터응용전공','컴퓨터정보처리',
'컴퓨터정보통신공학부','전산과학과','컴퓨터전자물리학과','컴퓨터정보학과','컴퓨터응용학과','컴퓨터시스템공학','컴퓨터교육과',
'서버,네트워크', '전산공학과','인터넷공학과','컴퓨터정보과학과','컴퓨터정보공학부','컴퓨터응용물리학과','소프트웨어학과',
'컴퓨터 산업공학과','전산정보학과','인터넷학과','International Summer Program','컴퓨터시스템과','컴퓨터응용과학',
'컴퓨터수리정보학과','컴퓨터게임','컴퓨터전자통신공학부','인터넷소프트웨어학과','컴퓨터 게임전공','컴퓨터전산통계학과',
'컴퓨터 인터넷 전공','컴퓨터전자공학과','통신컴퓨터공학부','인터넷정보공학','인터넷응용공학','네트워크공학',
]


호텔 = ['호텔경영', '호텔경영전공', '호텔경영학', 'HOTEL MANAGEMENT'
'호텔 관광 경영학과', '호텔 관광경영학과', '호텔.관광 경영학과', '호텔관관경영학과',
'호텔관광', '호텔관광 경영학과', '호텔관광경영', '호텔관광경영과', '호텔관광경영대학', '호텔관광경영학',
'호텔경영학과','호텔관광경영학과','호텔컨벤션','호텔관광대 외식경영학과','호텔 관광 경영학과',
'HOTEL MANAGEMENT','호텔조리','호텔관광학부',]


화학 = ['화학', '화학 전공', 'Chemistry','화학교육','화학공학과',
'화학공', '화학 전공','화학공학', '회힉공학','화학공학 경영학 부전공', '화학공항과', '화학공확과',
'응용화학', '응용화학학과','고분자', '고분자공(신소재공학)', '고분자공학', '고분자시스템공학과',
 '고분자시스템공학부', '고분자화학공학','공업화학','신소재공학', '신소재 공학', '신소재(고분자)공학', '신소재공학부','신소재 물리', '신소재 물리학', '신소재물리',
'신소재과', '신소재','세라믹', '세라믹공학', '세라믹공', '세라믹공학(무기재료)','재료공학', '재료공', '재료 공학',
 '재료공합','응용소재', '응용소재공학(구 금속공학)','화학,전기전자 복수전공','신소재공학과','재료/전자공학과','화학과',
 '무기재료','고분자공학과','재료공학과','재료.화학공학부','재료시스템공학과','재료금속공학','세라믹공학과',
'응용화학과','화공생명공학과','정밀화학신소재','화학신소재', '바이오케미컬','나노신소재공학과','농화학과',
'신소재학과','화학공학부','화공','생화학','화학교육과','천연물화학','공업화학과','신소재화학','응용소재공학과',
'정밀화학과','자동화공학과','금속', '금속 재료공학', '금속공', '금속공학', '금속공학과', '금속재료',
'금속재료공','금속재료공학과',]


환경 = ['환경', '환경과', '환경학','환경 과학과', '환경과학','환경공학', '환경공', '환경공학과(환경교육직이수)', 
 '환경공학전공','환경보건','환경보건학','환경생명공학','환경시스템공학', '환경시스템공학(활성탄열재생)',
 '환경식물공학', '환경식물공학과(구농학과)','환경화공', '환경화공공학부', '환경화공학',
'자원환경', '자원환경공학','환경공학과','자원식물학과', '자원식물학','자원식물','환경생명과학','식품환경화공학부',
'환경생명공학과','지구환경','환경보건학과','환경교육','자원환경공학과','인간생활환경학','환경학과','환경과학과',
'지구환경공학','지구시스템과학',]

섬유 = ['생활과학대학 천연섬유학과', '천연섬유', '천연섬유(섬유공학전공변경)', '천연섬유학',
'섬유공학', '섬유 공학과', '섬유','섬유공','섬유공학과 (텍스타일시스템공학)',
'의상섬유학부 섬유공학과', '텍스타일시스템공학','섬유공학/의상학','텍스타일','섬유예술과','섬유공학과',
'섬유예술학과','의류환경학과','의류/경영'
    ]

미술 = ['미술', '미술학', '미술전공','미술학부','동양화', '미술대학/동양화', '미술대학 동양화과', '미술학과 동양화전공', 
'회화과 동양화전공','미술대-서양화과', '미술학과(서양화전공)', '미술학과-서양화전공', '서양화', '서양화 전공',
'서양화(회화)', '서양화전공', '회화(서양화)', '회화과 서양화 전공','미술대학 조소학과', '미술대학/ 조소과',
 '예술대학/조소학과','미술학부 회화과', '회화', '회화학과','산업도예','고고미술사', '고고미술사학',
'원예', '원예학','원예과학','생활미술', '생활미술과', '생활미술학','조소', '조소학', '조소과', '조형예술학부 조소',
 '조형예술학부 조소과','조형예술','유학동양학','도예', '도자예술','조경', '조경학','만화애니메이션','한국화','원예학과',
'생활미술학과','미술학과','섬유예술','만화애니메이션과',
     '서양화과','목공예','미술대학','조소/멀티미디어','산업미술학과','조형예술학부','조소학과','동양화과','회화,판화과',
    '회화과','도예과','공예과','회화판화','공예(섬유,텍스타일 디자인)','큐레이터학','유학동양학과','조형학과', 
    '미술교육','영상예술공학','조경학과','장식미술학과','원예과학과','환경원예','일러스트','원예조경','영화예술학과',
    '섬유미술학과','공예학부 섬유예술과', '조각미술',
    ]


통계 = ['경제학부 통계학', '경제학부 통계학 전공', '경제학부 통계학과', '경제학부 통계학전공', '통계', 
 '통계(경영)','통계학','통계정보','응용정보통계', '응용통계', '응용통계(상경)', '응용통계학', 
 '경상대 응용통계', '응용통계학과(야간)','응용통계학과','통계학과','수학통계학','통계정보학과','전산통계학과',
      '계산통계','정보통계학과','전산통게'
     ]

산림 = ['산림자원', '산', '산림자원학','산림','산림자원학과',]

보험 = ['보험', '보험학','금융보험', '금융보험학', '금융선물보험학과','금융', '금융학','재무금융학과'
     ,'금융보험학과','금융보험학과','금융학과','보험학과','금융선물보헙학과','금융부동산학과']

주거 = ['주거환경', '주거환경학', '주거솬경학','부동산', '부동산학','주거환경학과','부동산학과','산업재산권학과'
    ]

사진 = ['사진', '사진학', '순수사진','사진영상','영상만화학', '영상만화','영화예술','영상', '영상학','영상만화학과',
     '사진학과','영상학과','사진영상학']

제약 = ['제약', '제약학전공', '제약학','약학','의예과','의과대학','제약공학과',]

음악 = ['음악', '음악학', 'Music Theory','작곡','국악','음악학부 피아노전공', '피아노','성악',
      '음악(성악)','음악교육','음악과','국악과','실용음악과','한국음악 거문고전공','성악과','작곡학과','음악대학 관현악과','기악과','기악(관악)',
    ]

윤리 = ['국민윤리교육', '국민윤리', '국민윤리학', '윤리교육','국민윤리', '국민윤리학','윤리학과','국민윤리학과']

무용 = ['무용', '무용전공', '무용학과','댄스스포츠',
     '한국무용', '무용과','현대무용','발레(무용)','동양무예학과']

신학 = ['기독교교육','기독교교육과','기독교교육학','기독교학','교회음악','선교학','신학',
      '신학부','통일신학','목회학','기독학/광고홍보학','신학/기독교교육학','신학과',
     '선교언어학','해외선교학','목회학과','기독교학과', '교회음악과','선교학과','기독교교육학과','통일신학과','선교영어학',
    ]

동물 = ['독물자원학', '동물자원학(축산학)', '동물자원', '동물영양자원', '동물자원과학','응용동물자원학과',
     '생자대 동물영양','동물자원학과','동물자원학','자연과학계열 동물과학학과'
    ]


과학 = ['생활과학', '생활과학과 졸업', '생활과학부','과학교육', '과학교육 화학전공', 
 '과학교육(생물)', '과학교육과', '과학교육과(생물전공)','새왈과학부','생활과학과',]

자연과학 = ['물리', '물리학', '물리락화','지구과학','지리', '지리학','지질학',
      '물리학전공,  컴퓨터공학부전공','물리,광고홍보','물리학과',
       '지리교육','과학교육학과','지질학과','물리교육학','지구과학과','천문우주학과','물리학전공,   컴퓨터공학부전공',
    '지리학과','식물학','자연과학부','지구정보과학과','천문학','열유체','대기과학'
    ]

수학 = ['수학', '수학전공', '수학학과','수학교육','응용수학','수학, 컴퓨터공학','수학/컴퓨터',
      '수학과','컴퓨터응용수학과','수학교육학','정보수학','(응용)수학','응용 수학과', '응용수학과',]

식량 = ['식량원예','식량자원학과', '식량자원공학과', '식품공학', '식품공', '식품공학과(현 생명공학과)', '해양과학대학식품공학',
 '식품미생물', '식품미생물공학','식품생물공학','식품영양', '식품영양학','한방식품영양학','식품영양학과','낙농식품공학과',
     '식량원예과','식품과학','식품.미생물','식품공학과', '식품생명공학','식품미생물공학과','식품생명자원학','식품생명자원학과',
    '식품과학과','한방식품영양학과','식품가공학','식품.생명자원','식품'
    ]

심리 = ['심리', '심리학','심리, 영문','심리학/사회복지','심리학과']

조리 = ['조리과학', '영양조리과학과','조리외식졍영','양식학','외식.조리','양식학과',
     '조리외식경영','조리과학과','외식사업']

지역 = ['지역개발', '지역개발학', '지역개발학(도시계획전공)','지역계획',
 '지역사회', '지역사회개발', '지역사회개발 학과', '지역사회개발학','지역계획학과',
     '지역사회개발학과' ,'지역사회과학부','지역개발학과'
    ]

철학 = ['철학','한국철학','동양철학','철학/경영학','철학/법학부전공','철학과',
     '동양철학과', '한국철학과','유교철학과','중국철학']

아동 = ['아동', '아동학', '인간개발학부 아동학과','아동가족학', '아동가족','아동복지','청소년학과',
 '아동복지학','청소년', '청소년학','유아교육', '유아교육학', '유아교육과','사범계 유아교육학', '사범대학유아교육과',
     '아동학과','아동복지학과','유아교육학과','보육학과','유아보육',]

체육 = ['체욱학과', '체육', '체육과', '체육학', '체육학전공', '체육학전공 4학년', '건강보건대학/체육학과','체육경영학',
'체육교육', '체육교육학','레저', '레저스포츠', '레져스포츠', '스포츠레저학과', '스포츠레저','사회체육', 
 '사회체육 학과', '사회체육과','사회체육학','태권도''특수체육', '특수체육학','유도', '유도과','유도학',
'격기', '격기학과','태권도학과','체육학과','사회체육학과','스포츠과학','체육교육학과','스포츠과학과','운동기능', 
      '경호안전','경호학과','격기지도학과','레저스포츠학과','체육경영학과','경기지도','운동기능학과',
      '스포츠산업학과','태권도','특수체육학과','유도학과','스포츠경영','체육교육과','검도','특수체육','헬스',
    '사회체육학부 스포츠마케팅 전공','스포츠과학부','스포츠레져학과']


축산 = ['축산', '축산학','농학','축산학과','낙농공학','농과대학','낙농학과','낙농자원식품'
    ]

토목 = ['토목 공학과', '토목공', '토목공하과', '토목공학','토목', '토옥','토목환경','토목환경공',
 '토목환경공학','토목환경공학부/환경공학전공','토목공학과','토목환경공학과','토목과']

해양 = ['해양공', '해양공학','해양과학','해양환경공학', '해양환경','해양학','해양생산학', 
      '해양생산','해양생산시스템학','해양학과','해양자원','해양생산시스템학전공','해양과학부',
     '해양정보과학','제어계측공학과','해양토목','해양생산과학부','해양환경공학과','해양생명의학과','기계선박해양공학부',
    '해양생산학과','수산증식과','해양토목환경전공','해양생명의학','해양공학과','해양자원육성','해양생명과학부',
    '해양생명개발',
    ]

역사 = ['사학', '사하과', '역사', '역사학', '역사학과', '인문(사학)', '인문학부 역사학과','국사', '국사학',
'문화재보존과학', '문화재학','역사 교육과', '역사교육','역사문화', '역사문화학','민속', '민속학','사학/행정','사학과',
'기록정보학','문화제보존학과','전통전파공','국사학과','역사문화학부','민속학과','인류학','문화재보존학과','북한학과',
'고고미술사학과','문화인류전공','고고인류학과','역사와 사상','한국학','역사교육과']

중퇴 = ['경영학중퇴', '관광개발학과 2年 수료','도자공예(자퇴)', '법학과(중퇴)', '전자과 자퇴',
'중국학과(중퇴)', '컴퓨터공학(중퇴)', '컴퓨터공학과 자퇴']

기타 = ['ㅇ', 'ㅇㅇ','예.체능','어학연수','경기지도학과','사','준사서','생약자원','1','기악','SOC'
     ,'철철','영','기타','해선','전공','주생활학','F-Horn','TESOL','리모델링','인문','사체','도서관학과',
     '졸업','사료','유학','임학과',]


기타공학 = ['공학','교통공학','데이터정보','데이터정보(구 전산통계)',
'자원공', '자원공학','에너지(구 원자력공학)','안전공학','한공우주', 
'항공우주공학과','요업공학과','토&#47791공학','안전공학과','매체공학','공학계열',
'Materials Engineering','자원공학과','항공우주','기관공학과','낙농공학과','교통공학과',
'생산시스템공학',]


디자인 = ['패션디자인산업학전공','패션디자인 산업학과','패션산업','패션산업학', '패션산업학부','패션디자인산업학과',
 '디자인학부 패션디자인과', '미술대학 패션디자인과', '의상섬유학부 패션디자인 전공',
'장식미술  패션디자인', '장식미술(패션)', '패션디자인과', '패션 디자인과', '패션 디자인학과',
'패션디자인', '패션디자인 2000.03~2008.02', '패션디자인 전공','패션디자인 학과',
'패션디자인전공', '패션디자인학부', 'fashoin Design'
,'공예', '공예디자인', '공예디자인과', '공예디자인학과', '공예학', '공예학과', '산업공예', '공예디자인(금속,섬유)',
'공예(금속공예)', '공예학과(금속전공)', '금속공예', '금속공예디자인',
 '산업디자인 금속디자인전공', '조형대학(금속공예과)', '금속디자인과','공예(섬유)학과', 
 '공예(섬유공예)', '공예학 -섬유', '섬유공예','섬유디자인', '텍스타일디자인', '섬유디자인 (공예과)', '섬유디자인(공예)',
'섬유공예디자인', '공예과(섬유디자인)',
'공예학과(섬유디자인)','공예(섬유, 텍스타일 디자인)','공예디자인(섬유)', '공예디자인학과 (섬유전공)', '산업디자인/텍스타일디자인',
'산업디자인학과(섬유전공)','산업<섬유디자인>', '산업디자인학부 텍스타일디자인전공',
'섬유미술', '섬유미술학', '예술대학섬유미술전공(수석졸업)', '공예학부 섬유미술과', '미술학사 (섬유공예)',
'공예학과 섬유미술전공','섬유예술, 섬유예술과','섬유조형디자인', '섬유조형디자인과','섬유패션디자인', '섬유패션디자인과'
,'비주얼 디자인', '비쥬얼디자인(시각)', '산업디자인 시각미디어전공','산업디자인(시각)',
'산업디자인(시각디자인 전공)', '산업디자인(시각디자인)', '산업디자인/시각디자인', 
'산업디자인과 시각디자인전공', '산업디자인학과(시각디자인전공)', '시각', '시각디자인', '시각디자인과', '시각디자인학',
'디자인학과 써피스디자인', '서피스디자인', '써피스디자인','써피스디자인과', '써피스디자인(텍스타일디자인)',
'디자인학부 산업디자인과', '디자인학부 산업지자인전공','산업 디자인','산업 공예 디자인과', '산업디자이', '산업디자인', 
'산업디자인 학과', '산업디자인(제품)', '산업디자인공학', '산업디자인공학과', '산업디자인과', '산업디자인학', 
'산업디자인학과 제품디자인', '산업디자인학부', '산업제품디자인','산업디자인(실내및제품디자인)',
'산업미술디자인', '산업미술학과(공예디자인 전공)','산업미술', '산업미술학',
'실내, 환경 디자인', '실내건축디자인', '실내디자인', '실내디자인과', '실내환경디자인',
'장식미술학과/실내환경디자인','스페이스디자인(환경디자인)', '스페이스디자인','환경디자인','환경디자인 전공','환경디자인과',
'환경디자인전공','미디어디자인학부 인테리어디자인전공', '인테리어', '인테리어디자인', '실내장식',
'미술학부 공예(목,가구디자인)과', '가구디자인','디자인', '디자인대학교', '디자인과', '디자인학과','제품디자인', 
 '제품디자인전공','미디어디자인', '미디어디자인학부','무대디자인','시각정보디자인', '시각커뮤니케이션 디자인과','장식미술(복식디자인)',
 '장식미술과','산업디자인학과','산업미술과','시각정보디자인학과','공업디자인','의상디자인학과',
'컴퓨터디자인','미디어디자인과','패션디자인학과','장신구 디자인','무대디자인과','시각디자인학과','실내디자인학과','디자인학부','복식디자인'
,'패션섬유디자인학과','생활디자인', '의류패션산업전공','영상디자인','콘텐츠디자인학과','섬유패션디자인학과',
'써피스디자인학과','공연예술학과 의상디자인','컴퓨터그래픽디자인학과','산업디자인과시각디자인전공','산업미술디자인과',
'산업디자인학과 시각디자인전공','산업디자인학과 멀티미디어디자인전공(주간)','영화영상멀티미디어공학','정보디자인학과',
'멀티미디어디자인','제품디자인과','섬유조형디자인학과','공간환경디자인'
]


동양 = ['한국어문학', '한국어문학부', '한국어문학학과','한국어문학부 한문학과', '한문학','문예창작','문예창작학과','문예창작(소설)',
'한문교육', '한문교육학', '한문교육과','국어국문학과','국문/신방','일본어', '일본어전공', '일본어학', '일본어과', '일어', '일어과','일어일본어','일본통상',
'일본학', '일본','일문','일어일본학', '일어일분학과','일어 일문학', '일어일문','일어일문학과',
 '일어일문과', '일어일문학', '일어일문학과(편입)', '일어일문확과','사범대학 일어교육과', '일본어교육', '일어교육',
'국문', '국문과', '국문학','국어교육', '국어교육학','국어국문', '국어국문과',
 '국어국문과(편입)졸업', '국어국문학', '국어국문학가','동양어문학',
'임학','중국어 학과', '중국어', '중국어과', '중국어전공', '중국어학', '중국어학과',
'중국어중국문학', '중어중문', '중어중문과', '중어중문학','중국학','중문', '중문과', '중문학',
'중어 중국학과', '중어중국학', '중어중국', '중국언어문화','국문,사회학','국어/전산','일어교육,영어교육','중국어, 법학',
 '증어중문','국어국문, 영어영문','국어국문,광고홍보','동양어문학부','노문학','중국문학','중문/사회','중어중문/동양사',
'중어중문학과','한문교육학과','중국학과','일본어학과','국어교육과','언어학과','일본학과',
'중어중문(교직이수)','국문학과','중어중국학과','어문학부','문예창장학과','일문학과','일어일본학과','국어국문학교',
'중어학과','국억국문학과']


서양 = ['프랑스어','프랑스어문','불어불문학과',
 '영문', '영문과','영문학','영문학 전공','영문학(부전공:경영학)','영미전공',
'영어','영어 전공','영어과','영어심화전공','영어학','영문영어', '영미언어문화', '영어 영문', '영어 영문과', 
 '영어 영문학', '영어 영문학과','영어영문','영어영문학과' ,
'영어영문과','영어영문학', '영어영문학(불문학 부전공)','영여영문', '영여영문학','영어교육', '영어교육과', '영어교육학',
'불어 불문학과', '러시아학','불어불문','불어불문과','불어불문학','불어', 
 '불어학','불문','불문학','불어교육학', '불어교육',
'스페인어 중남미학과', '스페인어중남미', '스페인어 중남미학'
,'노어노문','노어노문학', '러시아어문학과','노어러시아', '노어러시아학과','러시아','러시아어'
,'독어독문', '독어독문 학과', '독어독문과', '독어독문학', '독일어', '독일언어문화학','독문학','독어독문학과',
'독문, 심리', '독문(영문)','영문/경영','영어영문/사학','영어영문학/호텔관광경영학', '스페인어',
'프랑스어문학과','독문학과','English language & literature','영어학과','영문학과',
'프랑스어과','독문,심리','영어교육학과', '독어학','외국어','불어학과','Chinese Classics education',
'독일어교육','스페인어중남미학','러시아학과','프랑스 문화학과','영어영문학부','서양학과']


미디어 = ['디지털 컨텐츠', '디지털 컨텐츠학', '디지털 콘텐츠', '디지털 콘텐츠학',
'미디어디자인학부 디지컬컨텐츠학', '미디어정보학부 디지컬컨텐츠학', '언론정보학부 디지털콘텐츠학과',
'멀티미디어 컨텐츠','멀티미디어', '멀티미디어학부', '멀티미디어학과(現 디지털미디어학)', '멀티미디어학부(인터넷응요)' ,
'멀티미디어 공학','멀티미디어 정보처리','멀티미디어 정보통신','멀티미디어 정보통신공학과',
'멀티미디어공학','멀티미디어공학전공','멀티미디어정보처리','멀티미디어정보처리학과', 
'멀티미디어시스템공학', '멀티미디어정보통신','멀티미디어통신', '멀티미디어통신공학','다중매체','신문방송', '신문방송학',
'광고', '광고학','광고 디자인', '광고,시각','광고정보','언론 광고 학부', '언론홍보학과','언론영상', '언론영상학','언론정보 학과','언론정보학', '언론정보', 
 '사회과학부 언론정보', '언론정보학부','경영학부 광고.홍보학과','경영학부 광고홍보', '광고&PR', '광고/홍보',
 '광고PR', '광고홍보','광고홍보(편입)', '광고홍보학', '언론정보학부 광고홍보학 전공','광고홍보영상',
'문헌정보', '문헌정보학','문화컨테츠학과','방송연예','출판광고','광고홍보(복수전공)','문헌정보/심리','신문방송/광고홍보',
 '시각미디어','신문방송학과','광고홍보학과','문헌정보학과','언론영상학과','다중매체영상학',
'언론정보 광고홍보','디지털컨텐츠학과','미디어창작학과','언론정보학과','디지털콘텐츠학','디지털콘텐츠',
'멀티미디어학과','방송통신','멀티미디어공학과','전자it미디어공학과','미디어정보학부 디지털컨텐츠학','광고정보학과',
'디지털미디어','정보미디어학과','디지털콘텐츠학과','언론정보학부 인터넷미디어 전공','방송','방송영상문화학과',
'정보시스템, 멀티미디어','정치매체관리학전공','광고학과','출판광고학과','언론광고학과','매체','정치언론홍보학전공',
'방송연예학','정치미디어학과']

In [35]:
가정_키워드 = ['가정','가족','노인','소비자']
건강_키워드 = ['간호','건강','보건','구조','생리','병리']
건설_키워드 = ['건설','건축']
경영_키워드 = ['경영','세무','회계','마케팅','비서','유통','경상','비즈니스','MIS','상경','기획','사무','영업','판매']
경제_키워드 = ['경제','통상','무역']
관광_키워드 = ['관광']
교육_키워드 = ['교육','사범']
국제_키워드 = ['국제']
기계_키워드 = ['기계','기게','메카트로닉스','제어',]
도시_키워드 = ['도시']
법_키워드 = ['법','외교','정치','행정','경찰']
사회_키워드 = ['사회','복지']
산업_키워드 = ['산업','산업재산']
생명_키워드 = ['생명','미생물','생물']
연극_키워드 = ['연극','연기']
의류_키워드 = ['의류','직물','의복']
전기_키워드 = ['전기','전자','기술']
정보_키워드 = ['정보','지식','통신','모바일']
컴퓨터_키워드 = ['컴퓨터','소프트웨어','전산','인터넷','게임','서버','네트워크','그래픽','디지털','IT','기술']
호텔_키워드 = ['호텔','HOTEL']
화학_키워드 = ['화학','신소재','금속','화공','재료','세라믹','고분자','소재','과학']
환경_키워드 = ['환경']
섬유_키워드 = ['천연','섬유','텍스타일']
미술_키워드 = ['미술','회화','원예','예술','조소','도예','조형','만화','애니메이션','예술']
통계_키워드 = ['통계']
산림_키워드 = ['산','산림']
보험_키워드 = ['보험','금융']
주거_키워드 = ['주거','부동산']
사진_키워드 = ['사진','영상','영상학과']
제약_키워드 = ['의과','제약','의예','약']
음악_키워드 = ['음악','작곡','국악','피아노','성악']
윤리_키워드 = ['윤리']
무용_키워드 = ['무용','현대무용','발레','댄스','예술']
과학_키워드 = ['과학교육','생활과학','과학']
자연과학_키워드 = ['물리','지구','지리','식물','천문','과학']
수학_키워드 = ['수학','응용수학'] 
식량_키워드 = ['식량','식품']
신학_키워드 = ['신학','목회','기독교','선교','교회']
동물_키워드 = ['동물']
심리_키워드 = ['심리']
조리_키워드 = ['조리','영양','양식','한식','외식','일식']
지역_키워드 = ['지역','개발']
철학_키워드 = ['철학']
아동_키워드 = ['아동','유아','청소년']
체육_키워드 = ['체육','레저','운동','태권도','유도','헬스','스포츠']
축산_키워드 = ['축산','낙농','농']
토목_키워드 = ['토목']
해양_키워드 = ['해양']
역사_키워드 = ['역사','사학','문화재','인류','고고']
중퇴_키워드 = ['중퇴','자퇴']
기타공학_키워드 = ['자원','안전','항공','우주','기술']
디자인_키워드 = ['패션디자인','패션','공예디자인','텍스타일','디자인','산업디자인','의상','의복']
동양_키워드 = ['한국어','일어','중국어','중문','일문','국어국문','일본어','중국','일본','한국','문예']
서양_키워드 = ['영어','영문','프랑스어','불어','불문','스페인어','러시아어','노어','노문','독어','독일어','독문','독일']
미디어_키워드 = ['미디어','컨텐츠','매체','언론','광고','홍보','신문','방송','문화']
기타_키워드 = ['기타']

major_keyword_list = [가정_키워드,건강_키워드,건설_키워드,경영_키워드,경제_키워드,기타공학_키워드,
              관광_키워드,교육_키워드,국제_키워드,기계_키워드,도시_키워드,미디어_키워드,법_키워드,
              사회_키워드,산업_키워드,생명_키워드,연극_키워드,의류_키워드,전기_키워드,정보_키워드,
              컴퓨터_키워드,호텔_키워드,화학_키워드,환경_키워드,섬유_키워드,미술_키워드,
              디자인_키워드,통계_키워드,산림_키워드,보험_키워드,주거_키워드,사진_키워드,
              제약_키워드,음악_키워드,윤리_키워드,무용_키워드,신학_키워드,동물_키워드,과학_키워드
              ,자연과학_키워드,식량_키워드,수학_키워드,심리_키워드,조리_키워드,지역_키워드,철학_키워드,
              아동_키워드,체육_키워드,축산_키워드,토목_키워드,해양_키워드,역사_키워드,중퇴_키워드,기타_키워드
              ,서양_키워드,동양_키워드]

major_list = [가정,건강,건설,경영,경제,기타공학,
              관광,교육,국제,기계,도시,미디어,법,
              사회,산업,생명,연극,의류,전기,정보,
              컴퓨터,호텔,화학,환경,섬유,미술,
              디자인,통계,산림,보험,주거,사진,
              제약,음악,윤리,무용,신학,동물,과학,
              자연과학,식량,수학,심리,조리,지역,철학,
              아동,체육,축산,토목,해양,역사,중퇴,기타,
              서양,동양]

major_list_name = ['가정','건강','건설','경영','경제','기타공학',
                   '관광','교육','국제','기계','도시','미디어','법',
                   '사회','산업','생명','연극','의류','전기','정보',
                   '컴퓨터','호텔','화학','환경','섬유','미술',
                   '디자인','통계','산림','보험','주거','사진',
                   '제약','음악','윤리','무용','신학','동물','과학',
                   '자연과학','식량','수학','심리','조리','지역','철학',
                   '아동','체육','축산','토목','해양','역사','중퇴','기타',
                   '서양','동양']

In [36]:
X_train['전공분야'] = X_train['대학전공']
X_test['전공분야'] = X_test['대학전공']

for idx,major in enumerate(major_list):
    X_train['전공분야'].replace(major,major_list_name[idx],inplace = True)
    X_test['전공분야'].replace(major,major_list_name[idx],inplace = True)
    
    
all_major = 가정+건강+건설+경영+경제+기타공학+관광+교육+국제+기계+도시+미디어+법+사회+산업+생명+연극+의류+전기+정보+컴퓨터+호텔+화학+환경+섬유+미술+디자인+통계+산림+보험+주거+사진+제약+음악+윤리+무용+신학+동물+과학+자연과학+식량+수학+심리+조리+지역+철학+아동+체육+축산+토목+해양+역사+중퇴+기타+서양+동양

In [37]:
def major_etc1(x):
    global all_major,major_list,major_list_name,major_keyword_list
    
    if x not in all_major:
        for i in range(len(major_keyword_list)):
            for keyword in major_keyword_list[i]:
                if keyword in str(x):
                    x = major_list_name[i]
    else:
        x = x
    return x



X_test['전공분야'] = X_test['전공분야'].apply(major_etc1)
X_test['전공분야'] = X_test['전공분야'].apply(lambda x: '기타' if x not in major_list_name else x)

In [38]:
major_keyword_df = pd.DataFrame({'전공분야':major_list_name,'전공_키워드' : major_keyword_list})
X_train = pd.merge(X_train,major_keyword_df,on = '전공분야',how='left')
X_test = pd.merge(X_test,major_keyword_df,on = '전공분야',how='left')

In [39]:
X_train['대학전공_and_경력세분화'] = '세부직종: ' + X_train['전공분야'] + ' 근무경력: '+X_train['근무경력_세분화']
X_test['대학전공_and_경력세분화'] = '세부직종: ' + X_test['전공분야'] + ' 근무경력: '+X_test['근무경력_세분화']

#### 어학시험 범주 묶기

In [40]:
X_train.replace({'어학시험':{' ':'N'}}, inplace=True)
X_train.어학시험.fillna('N', inplace=True)

In [41]:
X_train['어학시험언어'] = 'N'

In [42]:
EN = ['TOEIC', 'TOEFL', 'OPIc', 'TEPS', 'IELTS', 'TOEIC Speaking', 'TOEFL-CBT', 'G-TELP', 'TOEFL-IBT', 'TOEFL-PBT', 'SEPT', 'G-ETAT', 'GRE', 'G-TELP(GLT)', 'TOEIC S&W', 'OPI']
JP = ['JLPT', 'JPT', 'JTRA'] 
CH = ['新HSK', 'HSK']
ETC = ['기타', '기타시험']
FR = ['DALF', 'DELF']
for i,j in enumerate(X_train['어학시험']):
    if j in EN:
        X_train['어학시험언어'][i] = 'EN'
    elif j in JP:
        X_train['어학시험언어'][i] = 'JP'
    elif j in CH:
        X_train['어학시험언어'][i] = 'CH'
    elif j in ETC:
        X_train['어학시험언어'][i] = 'ETC'
    elif j in FR:
        X_train['어학시험언어'][i] = 'FR'
        
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,근무지역_a,근무지역_b,근무지역_c,근무희망형태,단과대,ID,전공분야,전공_키워드,대학전공_and_경력세분화,어학시험언어
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0,NaN,"서울,경기,서울",성균관대학교,국문,N,無,...,서울,경기,서울,-1,인문대,0_성균관대학교_국문_nan_無_70.0,동양,"[한국어, 일어, 중국어, 중문, 일문, 국어국문, 일본어, 중국, 일본, 한국, 문예]",세부직종: 동양 근무경력: 0to2,N
1,경영·기획·회계·사무,사무·총무·법무,NaN,35,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,...,부산,서울,일본,1,경영경제대,35_신라대학교_관광경영_JLPT_無_nan,관광,[관광],세부직종: 관광 근무경력: 2to4,JP


In [43]:
X_test['어학시험언어'] = 'N'

In [44]:
EN = ['TOEIC', 'TOEFL', 'OPIc', 'TEPS', 'IELTS', 'TOEIC Speaking', 'TOEFL-CBT', 'G-TELP', 'TOEFL-IBT', 'TOEFL-PBT', 'SEPT', 'G-ETAT', 'GRE', 'G-TELP(GLT)', 'TOEIC S&W', 'OPI']
JP = ['JLPT', 'JPT', 'JTRA'] 
CH = ['新HSK', 'HSK']
ETC = ['기타', '기타시험']
FR = ['DALF', 'DELF']
X_test['어학시험언어']='N'
for i,j in enumerate(X_test['어학시험']):
    if j in EN:
        X_test['어학시험언어'][i] = 'EN'
    elif j in JP:
        X_test['어학시험언어'][i] = 'JP'
    elif j in CH:
        X_test['어학시험언어'][i] = 'CH'
    elif j in ETC:
        X_test['어학시험언어'][i] = 'ETC'
    elif j in FR:
        X_test['어학시험언어'][i] = 'FR'
        
X_test.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,근무지역_a,근무지역_b,근무지역_c,근무희망형태,단과대,ID,전공분야,전공_키워드,대학전공_and_경력세분화,어학시험언어
0,전문·교육·자격,외국어·번역·통역,"중국어,영어,강사,사무,micro office,호텔서비스",0,NaN,"서울,중국,캐나다",이화여자대학교,증어중문,TOEFL,無,...,서울,중국,캐나다,-1,인문대,0_이화여자대학교_증어중문_TOEFL_無_80.0,동양,"[한국어, 일어, 중국어, 중문, 일문, 국어국문, 일본어, 중국, 일본, 한국, 문예]",세부직종: 동양 근무경력: 0to2,EN
1,영업·판매·TM,해외영업,해외영업,0,NaN,"서울,,",중앙대학교,경영학과,,有,...,서울,,,-1,경영경제대,0_중앙대학교_경영학과_ _有_80.0,경영,"[경영, 세무, 회계, 마케팅, 비서, 유통, 경상, 비즈니스, MIS, 상경, 기...",세부직종: 경영 근무경력: 0to2,N


In [45]:
X_train['어학시험'].fillna('N',inplace = True)
X_train.loc[X_train['어학시험'] == ' ','어학시험'] = 'BLANK'
X_train['상위어학시험'] = '0'
for i in range(16570):
    X_train['상위어학시험'][i] = X_train['어학시험'].loc[i][:5]
X_train.loc[X_train['상위어학시험'] == '新HSK','상위어학시험'] = 'HSK'

list(X_train['상위어학시험'].value_counts().index[12:])

for i in list(X_train['상위어학시험'].value_counts().index[12:]):
    X_train.loc[X_train['상위어학시험'] == i,'상위어학시험'] = '기타'

X_test['어학시험'].fillna('N',inplace = True)
X_test.loc[X_test['어학시험'] == ' ','어학시험'] = 'BLANK'
X_test['상위어학시험'] = '0'
for i in range(11048):
    X_test['상위어학시험'][i] = X_test['어학시험'].loc[i][:5]
X_test.loc[X_train['상위어학시험'] == '新HSK','상위어학시험'] = 'HSK'

for i in list(X_test['상위어학시험'].value_counts().index[12:]):
    X_test.loc[X_test['상위어학시험'] == i,'상위어학시험'] = '기타'
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,근무지역_b,근무지역_c,근무희망형태,단과대,ID,전공분야,전공_키워드,대학전공_and_경력세분화,어학시험언어,상위어학시험
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0,NaN,"서울,경기,서울",성균관대학교,국문,N,無,...,경기,서울,-1,인문대,0_성균관대학교_국문_nan_無_70.0,동양,"[한국어, 일어, 중국어, 중문, 일문, 국어국문, 일본어, 중국, 일본, 한국, 문예]",세부직종: 동양 근무경력: 0to2,N,N
1,경영·기획·회계·사무,사무·총무·법무,NaN,35,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,...,서울,일본,1,경영경제대,35_신라대학교_관광경영_JLPT_無_nan,관광,[관광],세부직종: 관광 근무경력: 2to4,JP,JLPT


#### 근무지역 - 김종윤

In [46]:
def region(x):
    if x in ['서울']:
        x = '수도권_서울'
    elif x in ['경기']:
        x = '수도권_경기'
    elif x in ['인천']:
        x = '수도권_인천'
    elif x in ['강원']:
        x = '강원'
    elif x in ['경남','경북','대구','부산','울산']:
        x = '영남'
    elif x in ['광주','전남','전북']:
        x = '호남'
    elif x in ['충남','충북']:
        x = '호서'
    elif x in ['제주']:
        x = '제주'
    elif x in ['대만','러시아','말레이시아','방글라데시','싱가포르','인도','인도네시아','일본','중국','필리핀','홍콩']:
        x = '해외_유라시아'
    elif x in ['미국','캐나다']:
        x = '해외_아메리카'
    elif x in ['프랑스','해외']:
        x = '해외_기타'
    elif x == '':
        x = ''
    else: 
        x = '기타'
    return x

def new_region(x):
    x = set(sorted(map(region,x.split(','))))
    x.discard('')
    return tuple(x)


X_train['new_근무지역'] = X_train['근무지역'].apply(new_region)
X_test['new_근무지역'] = X_test['근무지역'].apply(new_region)

#### 근무지역 경험횟수 

In [47]:
X_train['근무지역_경험횟수'] = X_train['new_근무지역'].apply(lambda x: len(x))
X_test['근무지역_경험횟수'] = X_test['new_근무지역'].apply(lambda x: len(x))

### 세부직종

In [49]:
df_세부직종 = pd.read_csv('data/세부직종.csv', encoding='cp949')
df_세부직종.head()

,IT·게임,건설·기계·전기·전자,기술·과학·산업,영업·판매·TM,재료·화학·섬유·의복,전문·교육·자격,통신·모바일,기타
0,게임프로그래머,CAD·CAM,조선·우주·항공,해외영업,금속·금형·요업·재료·열처리,경영분석·컨설팅·M&A,네트워크·LAN,컴퓨터·IT·디자인강사
1,데이터베이스 전문직,건설·토목·건축설비,생명공학·유전공학,NaN,목재·가구·제지,금융·증권·투자,모바일프로그래머,경리·회계
2,시스템프로그래머,기계·기계설계·기계설비,연구개발·R&D,NaN,섬유·의복,법률·법무·특허·상표,모바일기획,비서·인포메이션·수행·안내
3,게임운영·테스터,기타 건설·기계·전기·전자,안전·시설관리,NaN,제약·의약,보험·연금,무선·위성·이동통신,사무보조·OA·업무보조
4,게임기획,반도체·디스플레이·광학,에너지·환경,NaN,화학·화공,부동산·경매·감정평가,유비쿼터스·와이브로·WCDMA,상담·고객지원


In [50]:
IT·게임 = list(df_세부직종['IT·게임'])
건설·기계·전기·전자 = list(df_세부직종['건설·기계·전기·전자'])
기술·과학·산업 = list(df_세부직종['기술·과학·산업'])
영업·판매·TM = list(df_세부직종['영업·판매·TM'])
재료·화학·섬유·의복 = list(df_세부직종['재료·화학·섬유·의복'])
전문·교육·자격 = list(df_세부직종['전문·교육·자격'])
통신·모바일 = list(df_세부직종['통신·모바일'])
기타 = list(df_세부직종['기타'])

In [51]:
X_train['세부직종_연봉'] = X_train['세부직종'].copy()
X_test['세부직종_연봉'] = X_test['세부직종'].copy()

In [52]:
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(IT·게임, 'IT·게임')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(건설·기계·전기·전자, '건설·기계·전기·전자')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(기술·과학·산업, '기술·과학·산업')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(영업·판매·TM, '영업·판매·TM')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(재료·화학·섬유·의복, '재료·화학·섬유·의복')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(전문·교육·자격, '전문·교육·자격')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(통신·모바일, '통신·모바일')
X_train['세부직종_연봉'] = X_train['세부직종_연봉'].replace(기타, '기타')

In [53]:
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(IT·게임, 'IT·게임')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(건설·기계·전기·전자, '건설·기계·전기·전자')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(기술·과학·산업, '기술·과학·산업')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(영업·판매·TM, '영업·판매·TM')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(재료·화학·섬유·의복, '재료·화학·섬유·의복')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(전문·교육·자격, '전문·교육·자격')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(통신·모바일, '통신·모바일')
X_test['세부직종_연봉'] = X_test['세부직종_연봉'].replace(기타, '기타')

### 소프트웨어대+공대

In [54]:
X_train['소프트웨어대+공대'] = X_train['단과대'].copy()
X_test['소프트웨어대+공대'] = X_test['단과대'].copy()

X_train = X_train.replace({'소프트웨어대+공대': {'소프트웨어대', '공대'}}, '공대')
X_train = X_train.replace({'소프트웨어대+공대': {'인문대', '사과대', '경영경제대', '교육대',
                          '예체능', '보건계열', '자연과학', '법대', '농축산대', '건축대', '신학대', '결측'}}, '공대X')

X_test = X_test.replace({'소프트웨어대+공대': {'소프트웨어대', '공대'}}, '공대')
X_test = X_test.replace({'소프트웨어대+공대': {'인문대', '사과대', '경영경제대', '교육대',
                                       '예체능', '보건계열', '자연과학', '법대', '농축산대', '건축대', '신학대', '결측'}}, '공대X')

### 대학+전공

In [55]:
X_train['대학+전공'] = X_train['출신대학'] + X_train['대학전공']
X_test['대학+전공'] = X_test['출신대학'] + X_test['대학전공']

### 근무형태+직종

In [56]:
X_train['근무형태+직종'] = X_train['근무형태']+X_train['직종']
X_test['근무형태+직종'] = X_test['근무형태']+X_test['직종']

### 출신대학*대학성적

In [57]:
top_rank = ['성균관대학교', '연세대학교', '중앙대학교', '이화여자대학교']
mid_rank = ['세종대학교', '성신여자대학교']
low_rank = ['한성대학교', '동덕여자대학교', '서울여자대학교', '서울과학기술대학교']
province = ['제주대학교', '인천대학교', '군산대학교', '한국산업기술대학교']
etc = ['수원대학교', '전주대학교', '세명대학교', '신라대학교', '상지대학교', '한밭대학교', '경기대학교', '선문대학교', '성공회대학교', '호원대학교', '한림대학교', '목포대학교', '동아대학교', '동의대학교',
       '성결대학교', '호남대학교', '광주대학교', '서원대학교', '동서대학교', '한일장신대학교', '한세대학교', '용인대학교', '경주대학교', '협성대학교', '서울신학대학교', '송원대학교', '부산디지털대학교', '남부대학교', '기타']


university = []
for i in X_train['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1)

X_train['출신대학수치'] = university

university = []
for i in X_test['출신대학']:
    if i in top_rank:
        university.append(5)
    elif i in mid_rank:
        university.append(4)
    elif i in low_rank:
        university.append(3)
    elif i in province:
        university.append(2)
    elif i in etc:
        university.append(1)

X_test['출신대학수치'] = university

a = X_train['출신대학수치'] * X_train['대학성적']
X_train['출신대학*대학성적'] = a
a = X_test['출신대학수치'] * X_test['대학성적']
X_test['출신대학*대학성적'] = a

X_train = X_train.drop(['출신대학수치'], axis=1)
X_test = X_test.drop(['출신대학수치'], axis=1)

In [58]:
# 근무지역_소분류
X_train['근무지역_소분류'] = X_train['근무지역'].copy()
X_test['근무지역_소분류'] = X_test['근무지역'].copy()

In [59]:
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [60]:
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [61]:
X_train.to_csv('X_train_Kyh.csv', encoding='cp949', index=False)
X_test.to_csv('X_test_Kyh.csv', encoding='cp949', index=False)
y_train.to_csv('y_train_Kyh.csv', encoding='cp949', index=False)

In [62]:
X_train.dtypes

직종                 object
세부직종               object
직무태그               object
근무경력                int64
근무형태               object
근무지역               object
출신대학               object
대학전공               object
어학시험               object
자격증                object
대학성적              float64
NaN_count           int64
NotNull_count       int64
숙련도                object
근무경력_y              int64
근무경력_m              int64
신입경력                int64
근무경력^2              int64
근무경력_세분화           object
마지막근무형태            object
해외근무지역             object
근무지역_a             object
근무지역_b             object
근무지역_c             object
근무희망형태              int64
단과대                object
ID                 object
전공분야               object
전공_키워드             object
대학전공_and_경력세분화     object
어학시험언어             object
상위어학시험             object
new_근무지역           object
근무지역_경험횟수           int64
세부직종_연봉            object
소프트웨어대+공대          object
대학+전공              object
근무형태+직종            object
출신대학*대학성적   